In [271]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import pandas as pd

In [272]:
# Carga de datos
train_input = np.asarray(pd.read_csv('train_data_input.csv', sep=',', header=None))

train_output = np.asarray(pd.read_csv('train_data_output.csv', sep=',', header=None))

test_input = np.asarray(pd.read_csv('test_data_input.csv', sep=',', header=None))

In [273]:
## Entrenamiento no supervisado con algoritmo K-means

# Agrupar puntos en clústers
k = 3 # Numero de neuronas en capa oculta
model = KMeans(k)
model.fit(train_input)
# Mostramos los puntos de los centroides (en un espacio R^8)
print("Centroides del modelo: \n" , model.cluster_centers_)
print("Iteraciones del K-Means: \n", model.n_iter_)

Centroides del modelo: 
 [[0.37985    0.27405    0.381      0.2925     0.83575    0.30755
  0.3336     0.19665   ]
 [0.2842     0.5405     0.43993333 0.75466667 0.7741     0.2353
  0.2222     0.4169    ]
 [0.805975   0.56155    0.23615    0.642125   0.432925   0.325025
  0.312175   0.523525  ]]
Iteraciones del K-Means: 
 3


In [274]:
## Entrenamiento supervisado

# Extraer centroides
c = model.cluster_centers_

# Calcular el sigma
sigma = (max(c.flatten())-min(c.flatten()))/np.sqrt(2*k)

x = train_input
# Calcular matriz G
p = 9 # Cantidad de muestras

G = np.zeros((p,k))
for i in range(p):
    for j in range(k):
        dist = np.linalg.norm(x[i]-c[j], 2) # Distancia euclideana Entre Xi y Cj
        G[i,j] = np.exp((-1/(sigma**2))*dist**2) # Resultado de la función de activación para Gij

W = np.dot(np.linalg.pinv(G), train_output)

print("Pesos: \n" , W)

Pesos: 
 [[ 4.31651156 -0.02933393 -0.03465584]
 [-0.03830551  0.11469198  1.74290534]
 [ 0.3752705   1.92746261  0.11004623]]


In [275]:
## Predicción de la red

x = test_input
p = 3 # 3 datos de prueba
G = np.zeros((p,k))
for i in range(p):
    for j in range(k):
        dist = np.linalg.norm(x[i]-c[j], 2) # Distancia euclideana Entre Xi y Cj
        G[i,j] = np.exp((-1/(sigma**2))*dist**2) # Resultado de la función de activación para Gij

ynew = np.dot(G, W) # Salida de la red
print(ynew)

[[1.05680143e+00 8.94126515e-04 4.70606814e-03]
 [1.86620134e-01 9.58798307e-01 5.67666831e-02]
 [1.51276776e-02 7.76830481e-02 4.45920142e-03]]


In [276]:
## Cálculo del Error

test_output = np.asarray(pd.read_csv('test_data_output.csv', sep=',', header=None))
#error = (test_output - ynew)**2
error = 0
for i in range(3):
    dist = np.linalg.norm(test_output[i]-ynew[i], 2)
    errorprom = dist/3
print("Error: ")
print(errorprom)
precision = (1-errorprom)*100
print("Precisión: ")
print(precision)

Error: 
0.33289387434918355
Precisión: 
66.71061256508165
